In [1]:
import json 
import pandas as pd

In [2]:
from spell_correction import preprocess

In [3]:
def preprocess_w(row):
    try:
        return ' '.join(preprocess(row.query))
    except : 
        pass

In [ ]:
file=pd.read_json('zappos-us_product_cleanData.json',lines=True)

In [ ]:
#file.head()

In [ ]:
#file.category.unique()

In [ ]:
#col=file[file['category']=='Clothing']
col=file

In [ ]:
col.gender.unique()

In [ ]:
col.groupby('color_slug').size().nlargest(10)

In [ ]:
def query_gen(row):
    try:
        qs=['show me best party {1}','show {0} {1}','best boat cut {1}','awesome meme {1} in trend now ',
            'combo of {1} {0} color ','comfortable {1} in range of {2}' ,'cute Baby One Pieces in color {0}']
        p=row.meta.split('>')[1]
        if 'shoes' in p.lower() and row.gender.lower == 'women':
            return qs[2].format(row.color_slug,'Heels')
        elif 'shoes' in p.lower() : 
            return qs[2].format(row.color_slug,'Boots')
        elif 'jeans' in p.lower():
            return qs[2].format(row.color_slug,p)
        elif 'hoodies'.lower() in p.lower() or 'Sweatshirts'.lower() in p.lower():
            return qs[3].format(row.color_slug,p)
        elif 'socks'.lower() in p.lower(): 
            return qs[4].format(row.color_slug,p)
        elif 'Sleepwear'.lower() in p.lower() or 'Jumpsuits'.lower() in p.lower():
            return qs[5].format(row.color_slug,p,row.mrp if row.mrp % 100 == 0 else row.mrp + 100 - row.mrp % 100)
        elif 'cute Baby'.lower() in p :
            return qs[6].format(row.color_slug,p)
        return qs[row.name%2].format(row.color_slug,p)
    except :
        pass

In [ ]:
#col['meta_n']=col.apply(meta_gen,axis=1)

In [ ]:
#col.groupby('meta_n').size().nlargest(20)

In [ ]:
col['query']=col.apply(query_gen, axis=1)

In [ ]:
col.groupby('query').size().nlargest(20)

In [ ]:
def meta_gen(row):
    try:
        return row.meta.split('>')[1]
    except:
        pass

In [ ]:
col['meta_n']=col.apply(meta_gen,axis=1)

In [ ]:
col.head()

In [ ]:
df_write=col[['category','brand','color_slug','gender','mrp','meta_n','query']]

In [ ]:
df_write.head()

In [ ]:
df_write.to_csv('query_intent.csv')

In [ ]:
df_new=pd.read_csv('query_intent.csv')

In [ ]:
#df_new.head(200)

In [ ]:
df_new['clean_query']=df_new.apply(preprocess_w,axis=1)

In [ ]:
df_new.head()

In [ ]:
#df_new['meta_n'].unique()

In [ ]:
cat_df=df_new.groupby(['clean_query','category'],as_index=False).size().reset_index(name='catc')

In [ ]:
b_df=df_new.groupby(['clean_query','brand']).size().reset_index(name='branc')

In [ ]:
c_df=df_new.groupby(['clean_query','color_slug']).size().reset_index(name='colc')

In [ ]:
p_df=df_new.groupby(['clean_query','meta_n']).size().reset_index(name='metc')

In [ ]:
t_df=df_new.groupby('clean_query').size().reset_index(name='tot')

In [ ]:
t_df.head()

In [ ]:
#p_df.head()

In [ ]:
#pd.merge(cat_df,b_df,on='clean_query')

In [ ]:
intent_df=cat_df.merge(b_df,on='clean_query').merge(c_df,on='clean_query').merge(p_df,on='clean_query')

In [ ]:
#intent_df.head()

In [ ]:
insight_df=intent_df[intent_df['catc'] != intent_df['branc']]

In [ ]:
insight_df=insight_df.merge(t_df,on='clean_query')

In [ ]:
insight_df.head()

In [ ]:
insight_df.to_csv('intent_analysis.csv')

In [29]:
insight_df=pd.read_csv('intent_analysis.csv')

In [30]:
#insight_df=insight_df[insight_df['metc'] != insight_df['tot']]

In [14]:
insight_df['b_in']=insight_df.apply(lambda x : round((x.branc/x.tot)*100,4),axis=1)
#insight_df=insight_df.sort_values('b_in',ascending=False).head(5)

In [15]:
insight_df['c_in']=insight_df.apply(lambda x : round((x.colc/x.tot)*100,4),axis=1)
#insight_df=insight_df.sort_values('c_in',ascending=False).head(5)

In [16]:
insight_df['m_in']=insight_df.apply(lambda x : round((x.metc/x.tot)*100,4),axis=1)
#insight_df=insight_df.sort_values('m_in',ascending=False).head(5)

In [34]:
insight_df['cat_in']=insight_df.apply(lambda x : round((x.catc/x.tot)*100,4),axis=1)

In [35]:
final_intent=insight_df[['clean_query','b_in','c_in','cat_in','m_in']]

In [32]:
insight_df.groupby(['clean_query','brand'],as_index=False)..head(2)

KeyError: 'b_in'

In [37]:
final_intent.clean_query.unique()

array(['ashcanvas loafer', 'black backpack', 'black belt', 'black boot',
       'black coat outerwear', 'black duffle bag', 'black fashion watch',
       'black flat', 'black handbag', 'black hat', 'black heel',
       'black loafer', 'black luggage', 'black pant', 'black sandal',
       'black shirt top', 'black short', 'black swimwear',
       'black underwear intimate', 'black wallet', 'blackcanvas loafer',
       'blackfullgrain boot', 'blackkidsuede heel', 'blackleather heel',
       'blackleather loafer', 'blacknubuck boot', 'blackpatent heel',
       'blacksuede heel', 'blackvachetta heel', 'boat cut boot',
       'boat cut jean', 'brown fashion watch', 'chestnut boot',
       'cognac belt', 'combo sock black color',
       'comfortable jumpsuit romper range', 'comfortable sleepwear range',
       'khaki shirt top', 'leopard sandal',
       'meme hoodies sweatshirt trend', 'natural heel', 'navy hat',
       'party backpack', 'party belt', 'party boot', 'party bracelet',
       '

In [38]:
final_intent.to_csv('intent.csv')

In [4]:
final_intent=pd.read_csv('intent.csv')

In [5]:
final_intent.head()

,Unnamed: 0,clean_query,brand,b_in,color_slug,c_in,category,cat_in,meta_n,m_in
0,0,ashcanvas loafer,TOMS,93.3333,ash-canvas,100.0,Shoes,100.0,Loafers,100.0
1,1,ashcanvas loafer,TOMS Kids,6.6667,ash-canvas,100.0,Shoes,100.0,Loafers,100.0
2,2,black backpack,Frye,20.0000,black,100.0,Bags,100.0,Backpacks,100.0
3,3,black backpack,Rebecca Minkoff,20.0000,black,100.0,Bags,100.0,Backpacks,100.0
4,4,black backpack,Tumi,60.0000,black,100.0,Bags,100.0,Backpacks,100.0


In [10]:
final_intent.groupby('clean_query').nlargest(5).reset_index(name='b_in').head(10)

TypeError: unhashable type: 'list'

In [6]:
final_intent[final_intent['clean_query'] == 'party dress'].head(100)

,Unnamed: 0,clean_query,brand,b_in,color_slug,c_in,category,cat_in,meta_n,m_in
309219,309219,party dress,Alternative,2.3256,black,9.3023,Clothing,100.0,Dresses,100.0
309220,309220,party dress,Alternative,2.3256,black-combo,2.3256,Clothing,100.0,Dresses,100.0
309221,309221,party dress,Alternative,2.3256,black-evening-sand,2.3256,Clothing,100.0,Dresses,100.0
309222,309222,party dress,Alternative,2.3256,black-white,2.3256,Clothing,100.0,Dresses,100.0
309223,309223,party dress,Alternative,2.3256,chutney,2.3256,Clothing,100.0,Dresses,100.0
309224,309224,party dress,Alternative,2.3256,coastal-blue-catch-n-keep-small,2.3256,Clothing,100.0,Dresses,100.0
309225,309225,party dress,Alternative,2.3256,coastal-blue-maybe-gator,2.3256,Clothing,100.0,Dresses,100.0
309226,309226,party dress,Alternative,2.3256,cream-combo,2.3256,Clothing,100.0,Dresses,100.0
309227,309227,party dress,Alternative,2.3256,eco-black,2.3256,Clothing,100.0,Dresses,100.0
309228,309228,party dress,Alternative,2.3256,gold,2.3256,Clothing,100.0,Dresses,100.0


In [41]:
temp.clean_query.unique()

array(['ashcanvas loafer', 'black backpack', 'black belt', 'black boot',
       'black coat outerwear', 'black duffle bag', 'black fashion watch',
       'black flat', 'black handbag', 'black hat', 'black heel',
       'black loafer', 'black luggage', 'black pant', 'black sandal',
       'black shirt top', 'black short', 'black swimwear',
       'black underwear intimate', 'black wallet', 'blackcanvas loafer',
       'blackfullgrain boot', 'blackkidsuede heel', 'blackleather heel',
       'blackleather loafer', 'blacknubuck boot', 'blackpatent heel',
       'blacksuede heel', 'blackvachetta heel', 'boat cut boot',
       'boat cut jean', 'brown fashion watch', 'chestnut boot',
       'cognac belt', 'combo sock black color',
       'comfortable jumpsuit romper range', 'comfortable sleepwear range',
       'khaki shirt top', 'leopard sandal',
       'meme hoodies sweatshirt trend', 'natural heel', 'navy hat',
       'party backpack', 'party belt', 'party boot', 'party bracelet',
       '

In [44]:
temp['query']=temp.apply(lambda x : ''.join(x.clean_query),axis =1 )

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [48]:
temp[['query']].to_csv('frequent_query.csv')

In [47]:
temp.groupby('query').size().nlargest(1000).reset_index(name='query_c')

,query,query_c
0,boat cut boot,305730
1,party heel,3710
2,party boot,2485
3,party sandal,1952
4,party shirt top,1060
5,party loafer,918
6,boat cut jean,640
7,party flat,612
8,party underwear intimate,580
9,party pant,360
